In [201]:
#imports
#load in standard data 
#load in models

from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
import pandas as pd
import sklearn
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score,classification_report
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
pd.set_option('display.max_columns', 500)
import pickle as pkl
vtc1 = pkl.load(open('vtc.pkl','rb'))
vtc2 = pkl.load(open('vtcpoly.pkl','rb'))
df1 = pd.read_csv('prepeddata.csv',index_col='Unnamed: 0')
df1['stimulant'] = df1['stimulant'].apply(lambda x: 'low' if x <= 2 else 'high')


stimtestdata = df1.drop(['hallucinagen','depressant'],axis = 1)
stimx = df1.drop('stimulant',axis = 1)
stimy = df1.stimulant



poly=PolynomialFeatures(degree=3,interaction_only=True)
scaler = StandardScaler()
polyscaler = StandardScaler()
selector = SelectKBest(k=43)

In [202]:
#traintest split default data 
stimXtrain,stimXtest,stimYtrain,stimYtest = train_test_split(stimx,stimy)
sm = SMOTE()
tl = TomekLinks()
sampling = SMOTETomek( random_state=None, smote=sm, tomek=tl, n_jobs=1)


stimXtrain, stimYtrain = sampling.fit_resample(stimXtrain, stimYtrain)

In [203]:
#fit std scaler to default data?

In [204]:
#make a pipeline of standard vtc
stdvtc = Pipeline([('scaler',scaler),('model', vtc1)])
stdvtc.fit(stimXtrain,stimYtrain)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 VotingClassifier(estimators=[('model1',
                                               LogisticRegression(C=0.1,
                                                                  class_weight='auto',
                                                                  max_iter=1000,
                                                                  solver='sag',
                                                                  warm_start=True)),
                                              ('model2',
                                               LogisticRegression(C=1,
                                                                  class_weight='auto',
                                                                  max_iter=1000,
                                                                  penalty='l1',
                                                                  solver='saga',
        

In [205]:
#instantiate d3 interaction only polyfeats

In [206]:
#instantiate select k best k =43

In [207]:
#instantiate standardscaler

In [208]:
#make a pipeline of poly vtc in order
# polyfeats
#select k best
#standardscaler
polyvtc = Pipeline([('poly',poly),('selector', selector),('scaler',polyscaler),('model', vtc2)])
polyvtc.fit(stimXtrain,stimYtrain)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Pipeline(steps=[('poly', PolynomialFeatures(degree=3, interaction_only=True)),
                ('selector', SelectKBest(k=43)), ('scaler', StandardScaler()),
                ('model',
                 VotingClassifier(estimators=[('polymodel1',
                                               LogisticRegression(C=0.5333333333333333,
                                                                  class_weight='auto',
                                                                  max_iter=1000,
                                                                  solver='sag',
                                                                  warm_start=True)),
                                              ('polymodel2',
                                               LogisticRegression(C=1,
                                                                  class_weight='auto',
                                                                  f...
                                            

In [209]:
# fit both

In [210]:
#check classification report for both
vtcypred = stdvtc.predict(stimXtest)
vtcytrain = stdvtc.predict(stimXtrain)

print(f1_score(vtcytrain,stimYtrain,average='weighted'),f1_score(stimYtest,vtcypred,average='weighted'))
print(classification_report(stimYtrain, vtcytrain))
print(classification_report(stimYtest, vtcypred))



confusion_matrix(stimYtrain, vtcytrain)

0.9523334325639472 0.8679151232267676
              precision    recall  f1-score   support

        high       0.92      0.98      0.95      1226
         low       0.98      0.92      0.95      1226

    accuracy                           0.95      2452
   macro avg       0.95      0.95      0.95      2452
weighted avg       0.95      0.95      0.95      2452

              precision    recall  f1-score   support

        high       0.52      0.75      0.61        71
         low       0.95      0.88      0.91       401

    accuracy                           0.86       472
   macro avg       0.74      0.81      0.76       472
weighted avg       0.89      0.86      0.87       472



array([[1207,   19],
       [  98, 1128]])

In [211]:
confusion_matrix(stimYtest,vtcypred)

array([[ 53,  18],
       [ 49, 352]])

In [212]:
#check classification report for both
vtcypred = polyvtc.predict(stimXtest)
vtcytrain = polyvtc.predict(stimXtrain)

print(f1_score(vtcytrain,stimYtrain,average='weighted'),f1_score(stimYtest,vtcypred,average='weighted'))
print(classification_report(stimYtrain, vtcytrain))
print(classification_report(stimYtest, vtcypred))



confusion_matrix(stimYtrain, vtcytrain)

0.9482465937882255 0.8640996554048737
              precision    recall  f1-score   support

        high       0.92      0.98      0.95      1226
         low       0.97      0.92      0.95      1226

    accuracy                           0.95      2452
   macro avg       0.95      0.95      0.95      2452
weighted avg       0.95      0.95      0.95      2452

              precision    recall  f1-score   support

        high       0.50      0.79      0.62        71
         low       0.96      0.86      0.91       401

    accuracy                           0.85       472
   macro avg       0.73      0.83      0.76       472
weighted avg       0.89      0.85      0.86       472



array([[1197,   29],
       [  98, 1128]])

In [213]:
confusion_matrix(stimYtest,vtcypred)

array([[ 56,  15],
       [ 55, 346]])

In [214]:
pkl.dump(stdvtc,open('finalstdvtc.pkl','wb'))
pkl.dump(polyvtc,open('finalpolyvtc.pkl','wb'))

In [215]:
finalvtc =VotingClassifier([('stdmodel',stdvtc),('polymodel',polyvtc)]
                           ,voting='soft',verbose = True,n_jobs=-1)
finalvtc.fit(stimXtrain,stimYtrain)




VotingClassifier(estimators=[('stdmodel',
                              Pipeline(steps=[('scaler', StandardScaler()),
                                              ('model',
                                               VotingClassifier(estimators=[('model1',
                                                                             LogisticRegression(C=0.1,
                                                                                                class_weight='auto',
                                                                                                max_iter=1000,
                                                                                                solver='sag',
                                                                                                warm_start=True)),
                                                                            ('model2',
                                                                             LogisticRegression(

In [216]:
vtcypred = finalvtc.predict(stimXtest)
vtcytrain = finalvtc.predict(stimXtrain)

print(f1_score(vtcytrain,stimYtrain,average='weighted'),f1_score(stimYtest,vtcypred,average='weighted'))
print(classification_report(stimYtrain, vtcytrain))
print(classification_report(stimYtest, vtcypred))



confusion_matrix(stimYtrain, vtcytrain)

0.9527396012191669 0.8779933002902027
              precision    recall  f1-score   support

        high       0.93      0.98      0.95      1226
         low       0.98      0.92      0.95      1226

    accuracy                           0.95      2452
   macro avg       0.95      0.95      0.95      2452
weighted avg       0.95      0.95      0.95      2452

              precision    recall  f1-score   support

        high       0.54      0.79      0.64        71
         low       0.96      0.88      0.92       401

    accuracy                           0.87       472
   macro avg       0.75      0.84      0.78       472
weighted avg       0.90      0.87      0.88       472



array([[1207,   19],
       [  97, 1129]])

In [217]:
confusion_matrix(stimYtest, vtcypred)

array([[ 56,  15],
       [ 47, 354]])

In [218]:
pkl.dump(polyvtc,open('finaloverallvtc.pkl','wb'))